# Fat-IBC transmission for bionic arm

This notebook contains the fat intrabody communication (fat-IBC) wireless transmission part of the bionic arm project.

We use Digi XBee radio modules which use the 802.15.4 protocol at an operating frequency of 2.45 GHz.
The XBee radio modules have external antenna connectors (U.FL. or SMA) which allow us to use antennas designed for fat-IBC through a phantom model.

Import the modules necessary to produce widgets:

In [1]:
from IPython.display import display
import ipywidgets as widgets

## 📲 Initialize the XBee radio module

The XBee radio modules and your computer communicate via serial over USB. 
We propose that you have only a single XBee radio plugged in at a time when you apply the configuration profiles.

> ℹ️ **NOTE:** Windows usually denotes the serial ports like `COMX`, where `X` will be an integer.
Linux serial ports are usually called `/dev/ttyUSBX` or `/dev/ttyACMX` in a similar fashion.

If you are unsure which serial port(s) are currently active on your computer, check the output of the following code:

In [2]:
import serial.tools.list_ports
ports = serial.tools.list_ports.comports()

for port, desc, hwid in sorted(ports):
        print("{}: {} [{}]".format(port, desc, hwid))

COM3: Standard Serial over Bluetooth link (COM3) [BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0002\7&62BB5F3&2&78A7EBAF44BC_C00000000]
COM4: Standard Serial over Bluetooth link (COM4) [BTHENUM\{00001101-0000-1000-8000-00805F9B34FB}_LOCALMFG&0000\7&62BB5F3&2&000000000000_00000000]
COM5: USB Serial Port (COM5) [USB VID:PID=0403:6015 SER=DN069WQRA]
COM6: USB Serial Device (COM6) [USB VID:PID=2341:0043 SER=75735303631351804171 LOCATION=1-2]


Specify which serial port the XBee is connected to:

In [3]:
XBEE_PORT = 'COM5'

Import the things we need from the XBee Python module and create an XBee device object with the serial port we defined earlier and a baud rate of 115200:

In [4]:
from digi.xbee.devices import XBeeDevice

xbee = XBeeDevice(XBEE_PORT, 115200)

Open the connection to the XBee device:

In [5]:
xbee.open()

Print the node name and MAC address of this XBee device:

In [6]:
print("Node ID: %s \t MAC: %s" % (xbee.get_node_id(), xbee.get_64bit_addr()) )

Node ID: TRANSMITTER 	 MAC: 0013A20041CB9200


Try to find the receiver/aggregator node on the network so we can send data to it later:

In [7]:
from digi.xbee.devices import RemoteXBeeDevice, XBee64BitAddress        

# Check the MAC address of your particular RX XBee!
AGGREGATOR_MAC = "0013A20041C14D4A" # the MAC address is printed on the back of the XBee
aggregator_xbee = RemoteXBeeDevice(xbee, XBee64BitAddress.from_hex_string(AGGREGATOR_MAC))     

## ➡️ Construct payload from neural data

Let us start by initializing an empty payload that we can populate with data:

In [8]:
payload = bytearray()

We want to use classified neural data to decide which payload to send.

In [9]:
#<CODE TO POPULATE PAYLOAD WITH DATA CORRESPONDING TO CHOSEN CLASS>

## 📡 Send 802.15.4 packet with XBee radio module

In [10]:
# tx_status = xbee.send_data(aggregator_xbee, bytearray([255] * 7))

# print(tx_status)

## 🖱️ Test buttons

Here are some buttons which may be useful when troubleshooting or for trying things out.

Should we always rotate the wrist? 
Could keep it always at the same value, maybe that works better...

In [15]:
from time import sleep

In [21]:
sleep(5)

# close pinky
xbee.send_data(aggregator_xbee, bytearray([0, 0, 0, 0, 255, 0, 0]))

sleep(2)
# close ring
xbee.send_data(aggregator_xbee, bytearray([0, 0, 0, 255, 255, 0, 0]))

sleep(2)
# close rest
xbee.send_data(aggregator_xbee, bytearray([255, 255, 255, 255, 255, 0, 0]))

sleep(2)
xbee.send_data(aggregator_xbee, bytearray([0, 0, 0, 255, 255, 0, 0]))

sleep(2)
xbee.send_data(aggregator_xbee, bytearray([0, 0, 0, 0, 255, 0, 0]))

sleep(2)
xbee.send_data(aggregator_xbee, bytearray([0]*7))

In [9]:
def set_0(val):
    xbee.send_data(aggregator_xbee, bytearray([0] * 7))

def set_128(val):
    xbee.send_data(aggregator_xbee, bytearray([128] * 7))
    
def set_255(val):
    xbee.send_data(aggregator_xbee, bytearray([255] * 7))
    
set_all_0 = widgets.Button(description = 'Set all 0')   
set_all_0.on_click(set_0)

set_all_128 = widgets.Button(description = 'Set all 128')   
set_all_128.on_click(set_128)

set_all_255 = widgets.Button(description = 'Set all 255')   
set_all_255.on_click(set_255)

display(set_all_0)
display(set_all_128)
display(set_all_255)a

Button(description='Set all 0', style=ButtonStyle())

Button(description='Set all 128', style=ButtonStyle())

Button(description='Set all 255', style=ButtonStyle())